<a href="https://colab.research.google.com/github/pointonjoel/MSc-Diss/blob/main/code/BERT_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
# !pip install torch
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.7 MB/s eta 0:00:00


In [3]:
model = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")#.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [4]:
tokenizer = BertTokenizer.from_pretrained("deepset/bert-base-cased-squad2")#.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
question = "When did universities begin teaching Computer Vision?"
paragraph = "In the late 1960s, computer vision began at universities that were pioneering artificial intelligence. It was meant to mimic the human visual system, as a stepping stone to endowing robots with intelligent behavior.[10] In 1966, it was believed that this could be achieved through a summer project, by attaching a camera to a computer and having it \"describe what it saw\"."

encoding = tokenizer.encode_plus(text=question,text_pair=paragraph)
inputs = encoding['input_ids']  #Token embeddings
sentence_embedding = encoding['token_type_ids']  #Segment embeddings
tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

In [ ]:
outputs = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
start_scores, end_scores = outputs.start_logits, outputs.end_logits

# Highlight the answer just by looking at the most probable start and end words
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)
answer_token_list = tokens[start_index:end_index+1]

# Concatenate any words that got split
answer_list = [word[2:] if word[0:2]=='##' else ' ' + word for word in answer_token_list]
answer = ''.join(answer_list).strip()

print(answer)


KeyboardInterrupt



In [ ]:
tokenized_input = tokenizer.tokenize(paragraph)
len(tokenized_input)

75

In [ ]:
from scipy import spatial  # for calculating vector similarities for search
def similarity(query_embedding: list,
                   knowledge_embedding: list
    ) -> float:
        """Calculates the cosine similarity score between the query and knowledge embedding vectors."""

        return 1- spatial.distance.cosine(query_embedding, knowledge_embedding)

In [ ]:
from sentence_transformers import SentenceTransformer
similarity_model = SentenceTransformer('bert-base-nli-mean-tokens')
paragraph = "Joel speaks a load of rubbish"
question_encoding = similarity_model.encode(question)
paragraph_encoding = similarity_model.encode(paragraph)

In [ ]:
similarity(question_encoding, paragraph_encoding)

0.13547949492931366

In [ ]:
tokenizer.decode(tokenizer.encode(question))

'[CLS] When did universities begin teaching Computer Vision? [SEP]'

In [ ]:
question_encoding

[101, 2043, 2106, 5534, 4088, 4252, 3274, 4432, 1029, 102]

In [ ]:
len(paragraph_encoding)

768